In [9]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [26]:


current_directory = os.getcwd()
folder_name = 'train'
directory = os.path.join(current_directory, folder_name)
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        patient_num = os.path.splitext(filename)[0]
        #train_data[patient_num] = pd.read_excel(os.path.join(directory, filename), )


# Load the XML file
tree = ET.parse('train/100.xml')
root = tree.getroot()

overall_df = pd.DataFrame(columns=["note1", "note2", "note3", "note4", "note5", "ABDOMINAL",
"ADVANCED-CAD",
"ALCOHOL-ABUSE"
"ASP-FOR-MI",
"CREATININE",
"DIETSUPP-2MOS",
"DRUG-ABUSE",
"ENGLISH",
"HBA1C",
"KETO-1YR",
"MAJOR-DIABETES",
"MAKES-DECISIONS",
"MI-6MOS"])
# Access elements and attributes
for child in root:
    print("Child Tag:", child.tag)
    if child.tag == "TEXT":
        notes = child.text.split("****************************************************************************************************")
        notes = [note.strip() for note in notes if note.strip()]
        for note in notes:
            print("START OF NOTE")
            print(note)
    if child.tag == "TAGS":
        for subchild in child:
            print(subchild.tag)
            met_property = subchild.attrib.get('met')

Child Tag: TEXT
START OF NOTE
Record date: 2106-02-12

Campbell Orthopedic Associates
4 Madera Circle
Omak, GA 28172
 
Habib Valenzuela, M.D.
 
 
                                             Valdez, Harlan Jr.  
                                           845-41-54-4
                                             February 12, 2106 
Har is a 43 year old 6' 214 pound gentleman who is referred for
consultation by Dr. Harlan Oneil.  About a week ago he slipped on
the driveway at home and sustained an injury to his left ankle. 
He was seen at Tri-City Hospital and was told he had a
fracture.  He was placed in an air splint and advised to be
partial weight bearing, and he is using a cane.  He is here for
routine follow-up. 
Past medical history is notable for no ankle injuries previously. 
He has a history of diabetes and sleep apnea.  He takes Prozac,
Cardizem, Glucophage and Amaryl.  He is also followed by Dr. Harold
Nutter for an arrhythmia.  He does not smoke.  He drinks
minimally.  He is a